# Representing Text

We can represent text in many ways: character strings are a standard representation, but we can also create numerical representations of text. In this notebook we will discuss embeddings.

## Features

Features to any (machine learning) model can be continuous or categorical.

- We use continuous features to represent numerical values: income, number of times the user clicked on a link, prices, etc.
- Categorical features represent an instance of a class or category. They have a finite number of possible values: job title, genre of a movie, breed of a dog, etc.

## Embeddings

An **embedding** is a trained numerical representation of a categorical feature:

- We use the word *trained* to highlight that embeddings are learned during model training.
- Different models and training procedures can be used to obtained embeddings. Word2Vec and BERT embeddings, for example, are different and capture different characteristics of the features.

[OpenAI's documentation](https://platform.openai.com/docs/guides/embeddings) include a few uses of embeddings:


- Search: results are ranked by relevance to a query string.
- Clustering:  text strings are grouped by similarity.
- Recommendations:  items with related text strings are recommended.
- Anomaly detection:  outliers with little relatedness are identified.
- Diversity measurement: similarity distributions are analyze.
- Classification: text strings are classified by their most similar label.


## Tokenization

Most embedding computations start with the process of tokenization. Specifically, we can use the [`transformers`](https://huggingface.co/docs/transformers/en/index) library from [HuggingFace](https://huggingface.co/).

In [11]:
import transformers



documents = ["cats are fun"]

tokenizer = transformers.BertTokenizer.from_pretrained(
    'bert-base-uncased')
tokens = tokenizer(documents)
print(tokens)


{'input_ids': [[101, 8870, 2024, 4569, 102]], 'token_type_ids': [[0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


In the code snippet above, we used the `transformers` library to obtain the tokens that represent the phrase 'cats are fun'. The tokenizer returns a dictionary with an entry called `'input_ids'`, which contains an array (tensor if we were working on TensorFlow) of four integers.

In [1]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [ ]:
import os
from openai import OpenAI
import pandas as pd
import numpy as np

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables")
else:
    client = OpenAI(api_key = OPENAI_API_KEY)

In [ ]:
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:
documents = [
    # Freedom
    "Freedom consists not in doing what we like, but in having the right to do what we ought.",
    "Those who deny freedom to others deserve it not for themselves.",
    "Liberty, when it begins to take root, is a plant of rapid growth.",
    "Freedom lies in being bold.",
    "Is freedom anything else than the right to live as we wish?",
    "I am no bird and no net ensnares me: I am a free human being with an independent will.",
    "The secret to happiness is freedom... And the secret to freedom is courage."
    "Freedom is the oxygen of the soul.", 
    "Life without liberty is like a body without spirit."
    # Friendship
    "There is nothing on this earth more to be prized than true friendship.",
    "There are no strangers here; Only friends you haven’t yet met.",
    "Friendship is the only cement that will ever hold the world together.",
    "A true friend is someone who is there for you when he'd rather be anywhere else.",
    "Friendship is the golden thread that ties the heart of all the world.", 
    "Your friend is the man who knows all about you and still likes you.",
    "A single rose can be my garden... a single friend, my world."
]

In [ ]:
embeddings = [get_embedding(doc) for doc in documents]

In [ ]:
embeddings

In [ ]:
embeddings_array = np.array(embeddings)
embeddings_array

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
reduced_embeddings = pca.fit_transform(embeddings_array)

In [ ]:
reduced_embeddings

In [ ]:
df = pd.DataFrame(reduced_embeddings, columns=["x", "y"]).assign(lables = documents)
ax = df.plot(kind='scatter', x='x', y='y', figsize=(8, 6))
for i, row in df.iterrows():
    ax.text(row['x'], row['y'], row['lables'], fontsize=9)